In [99]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aviachipta-narxini-bashorat-qilish/train_data.csv
/kaggle/input/aviachipta-narxini-bashorat-qilish/test_data.csv
/kaggle/input/aviachipta-narxini-bashorat-qilish/sample_solution.csv


In [100]:
data = pd.read_csv("/kaggle/input/aviachipta-narxini-bashorat-qilish/train_data.csv",index_col='id')
data

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
id,,,,,,,,,,,
1,Vistara,UK-810,Bangalore,Early_Morning,one,Night,Mumbai,Economy,14.25,21,7212
2,SpiceJet,SG-5094,Hyderabad,Evening,zero,Night,Kolkata,Economy,1.75,7,5292
3,Vistara,UK-846,Bangalore,Morning,one,Evening,Delhi,Business,9.58,5,60553
4,Vistara,UK-706,Kolkata,Morning,one,Evening,Hyderabad,Economy,6.75,28,5760
5,Indigo,6E-5394,Chennai,Early_Morning,zero,Morning,Mumbai,Economy,2.00,4,10712
...,...,...,...,...,...,...,...,...,...,...,...
19996,Indigo,6E-6178,Bangalore,Night,one,Early_Morning,Mumbai,Economy,7.92,45,3153
19997,AirAsia,I5-582,Kolkata,Morning,one,Afternoon,Delhi,Economy,5.83,24,3911
19998,Vistara,UK-832,Chennai,Early_Morning,two_or_more,Evening,Bangalore,Economy,35.33,17,14822


# data analysis

In [101]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20000 entries, 1 to 20000
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   airline           20000 non-null  object 
 1   flight            20000 non-null  object 
 2   source_city       20000 non-null  object 
 3   departure_time    20000 non-null  object 
 4   stops             20000 non-null  object 
 5   arrival_time      20000 non-null  object 
 6   destination_city  20000 non-null  object 
 7   class             20000 non-null  object 
 8   duration          20000 non-null  float64
 9   days_left         20000 non-null  int64  
 10  price             20000 non-null  int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 1.8+ MB


In [102]:
data.describe()

,duration,days_left,price
count,20000.000000,20000.000000,20000.000000
mean,12.177627,25.924150,20960.281700
std,7.157944,13.624874,22775.459535
min,0.830000,1.000000,1105.000000
25%,6.830000,14.000000,4783.000000
50%,11.250000,26.000000,7425.000000
75%,16.080000,38.000000,42521.000000
max,38.580000,49.000000,114523.000000


In [103]:
data[['duration','days_left','price']].corr()

,duration,days_left,price
duration,1.000000,-0.020091,0.213158
days_left,-0.020091,1.000000,-0.102545
price,0.213158,-0.102545,1.000000


# preparing the data for the model

In [104]:
data['duration_pro'] = pd.cut(data['duration'], bins=[0., 4.0, 17.0, 28.0, np.inf], labels=['one','two','tree',"four"])

In [105]:
airline = pd.get_dummies(data['airline'])

data = data.drop('airline',axis=1)

data = pd.concat([data,airline],axis=1)

In [106]:
source_city = pd.get_dummies(data['source_city'])

data = data.drop('source_city',axis=1)

data = pd.concat([data,source_city],axis=1)

In [107]:
stops = pd.get_dummies(data['stops'])

data = data.drop('stops',axis=1)

data = pd.concat([data,stops],axis=1)

In [108]:
departure_time = pd.get_dummies(data['departure_time'])

data = data.drop('departure_time',axis=1)

data = pd.concat([data,departure_time],axis=1)

In [109]:
arrival_time = pd.get_dummies(data['arrival_time'])

data = data.drop('arrival_time',axis=1)

data = pd.concat([data,arrival_time],axis=1)

In [110]:
duration_pro = pd.get_dummies(data['duration_pro'])

data = data.drop('duration_pro',axis=1)

data = pd.concat([data,duration_pro],axis=1)

In [111]:
destination_city = pd.get_dummies(data['destination_city'])

data = data.drop('destination_city',axis=1)

data = pd.concat([data,destination_city],axis=1)

In [112]:
from sklearn.preprocessing import OrdinalEncoder
ordinal= OrdinalEncoder()

data['flight'] = ordinal.fit_transform(data[['flight']])
data['class'] = ordinal.fit_transform(data[['class']])

In [113]:
data = data.replace({False:0,True:1})

/tmp/ipykernel_34/444660677.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.replace({False:0,True:1})


In [114]:
from sklearn.model_selection import StratifiedShuffleSplit
stratified_split = StratifiedShuffleSplit(n_splits=1,test_size=0.2,random_state=42)
for train_index,test_index in stratified_split.split(data,data['class']):
  train_set = data.iloc[train_index]
  test_set =data.loc[test_index]

In [115]:
X = train_set.drop(['price'],axis=1)
y = train_set['price'].copy()

# model

In [116]:
from sklearn.ensemble import RandomForestRegressor
RFR_model = RandomForestRegressor(n_estimators=500,min_samples_split=3)

RFR_model.fit(X,y)

RandomForestRegressor(min_samples_split=3, n_estimators=500)

# model checking

In [117]:
X_test=test_set.drop('price',axis=1)
y_test=test_set['price'].copy()


In [118]:
y_predict = RFR_model.predict(X_test)

In [119]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test,y_predict)**(1/2)
mse

1908.3585458515413